# Bond Market Amount Checker

## Purpose
Verify market_amount data for individual bonds by comparing:
- Market Amount (from bond_data table)
- BOJ Holdings (from boj_holdings table)
- Cumulative Issuance (from bond_auction table)

## How to Use
1. Change the BOND_CODE in the cell below
2. Run all cells (Cell → Run All)
3. Check the table and chart output

## Setup

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dotenv import load_dotenv
from supabase import create_client

# Plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (16, 8)

# Supabase connection
load_dotenv()
url = os.getenv('SUPABASE_URL')
key = os.getenv('SUPABASE_KEY')
supabase = create_client(url, key)

print(f"✅ Connected to Supabase")

## 🔹 CHANGE BOND CODE HERE

In [ ]:
# ===== CHANGE THIS BOND CODE =====
BOND_CODE = '003720067'  # 10-Year Bond #372
# ==================================

## Data Retrieval

In [ ]:
print(f"Fetching data for bond code: {BOND_CODE}")
print("=" * 60)

# 1. Get market_amount from bond_data
result_bond = supabase.table('bond_data') \
    .select('trade_date, market_amount, bond_name') \
    .eq('bond_code', BOND_CODE) \
    .order('trade_date') \
    .execute()

df_bond = pd.DataFrame(result_bond.data)
print(f"✅ bond_data: {len(df_bond)} records")

# 2. Get BOJ holdings from boj_holdings
result_boj = supabase.table('boj_holdings') \
    .select('data_date, face_value') \
    .eq('bond_code', BOND_CODE) \
    .order('data_date') \
    .execute()

df_boj = pd.DataFrame(result_boj.data)
print(f"✅ boj_holdings: {len(df_boj)} records")

# 3. Get issuance from bond_auction
result_auction = supabase.table('bond_auction') \
    .select('auction_date, allocated_amount') \
    .eq('bond_code', BOND_CODE) \
    .order('auction_date') \
    .execute()

df_auction = pd.DataFrame(result_auction.data)
print(f"✅ bond_auction: {len(df_auction)} records")

print("=" * 60)

## Data Processing

In [ ]:
# Convert dates to datetime
if not df_bond.empty:
    df_bond['trade_date'] = pd.to_datetime(df_bond['trade_date'])
    df_bond['market_amount'] = pd.to_numeric(df_bond['market_amount'], errors='coerce')
    df_bond = df_bond.rename(columns={'trade_date': 'date'})

if not df_boj.empty:
    df_boj['data_date'] = pd.to_datetime(df_boj['data_date'])
    df_boj['face_value'] = pd.to_numeric(df_boj['face_value'], errors='coerce')
    df_boj = df_boj.rename(columns={'data_date': 'date', 'face_value': 'boj_holdings'})

if not df_auction.empty:
    df_auction['auction_date'] = pd.to_datetime(df_auction['auction_date'])
    df_auction['allocated_amount'] = pd.to_numeric(df_auction['allocated_amount'], errors='coerce')
    # Calculate cumulative issuance
    df_auction['cumulative_issuance'] = df_auction['allocated_amount'].cumsum()
    df_auction = df_auction.rename(columns={'auction_date': 'date'})

# Merge all data
# Use bond_data as base and merge others
if not df_bond.empty:
    df_merged = df_bond[['date', 'market_amount']].copy()
    
    # Merge BOJ holdings (forward fill to match bond_data dates)
    if not df_boj.empty:
        df_merged = df_merged.merge(
            df_boj[['date', 'boj_holdings']], 
            on='date', 
            how='left'
        )
        df_merged['boj_holdings'] = df_merged['boj_holdings'].fillna(method='ffill')
    
    # Merge cumulative issuance (forward fill to match bond_data dates)
    if not df_auction.empty:
        df_merged = df_merged.merge(
            df_auction[['date', 'cumulative_issuance']], 
            on='date', 
            how='left'
        )
        df_merged['cumulative_issuance'] = df_merged['cumulative_issuance'].fillna(method='ffill')
    
    # Sort by date
    df_merged = df_merged.sort_values('date').reset_index(drop=True)
    
    print(f"✅ Merged data: {len(df_merged)} records")
    print(f"Date range: {df_merged['date'].min()} to {df_merged['date'].max()}")
else:
    print(f"❌ No data found for bond code: {BOND_CODE}")
    df_merged = pd.DataFrame()

## 📊 Table View (First 20 rows)

In [ ]:
if not df_merged.empty:
    # Format for display
    df_display = df_merged.copy()
    df_display['date'] = df_display['date'].dt.strftime('%Y-%m-%d')
    
    # Round numbers for readability
    for col in ['market_amount', 'boj_holdings', 'cumulative_issuance']:
        if col in df_display.columns:
            df_display[col] = df_display[col].round(2)
    
    print(f"\n📋 First 20 rows of {len(df_display)} total records:")
    print("=" * 80)
    display(df_display.head(20))
    
    print(f"\n📋 Last 20 rows:")
    print("=" * 80)
    display(df_display.tail(20))
else:
    print("No data to display")

## 📈 Line Chart

In [ ]:
if not df_merged.empty:
    fig, ax = plt.subplots(figsize=(16, 8))
    
    # Plot market_amount
    if 'market_amount' in df_merged.columns:
        ax.plot(df_merged['date'], df_merged['market_amount'], 
                'b-', linewidth=2, label='Market Amount', marker='o', markersize=3)
    
    # Plot BOJ holdings
    if 'boj_holdings' in df_merged.columns:
        ax.plot(df_merged['date'], df_merged['boj_holdings'], 
                'r-', linewidth=2, label='BOJ Holdings', marker='s', markersize=3)
    
    # Plot cumulative issuance
    if 'cumulative_issuance' in df_merged.columns:
        ax.plot(df_merged['date'], df_merged['cumulative_issuance'], 
                'g-', linewidth=2, label='Cumulative Issuance', marker='^', markersize=3)
    
    # Chart formatting
    bond_name = df_bond['bond_name'].iloc[0] if not df_bond.empty else BOND_CODE
    ax.set_title(f'{bond_name} ({BOND_CODE}) - Time Series Comparison', 
                fontsize=14, fontweight='bold')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Amount (100M JPY)', fontsize=12)
    ax.legend(loc='best', fontsize=11)
    ax.grid(True, alpha=0.3)
    
    # Add zero line
    ax.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Chart displayed above")
else:
    print("No data to plot")

## 📈 Summary Statistics

In [ ]:
if not df_merged.empty:
    print("\n📊 Summary Statistics:")
    print("=" * 80)
    
    for col in ['market_amount', 'boj_holdings', 'cumulative_issuance']:
        if col in df_merged.columns:
            print(f"\n{col.replace('_', ' ').title()}:")
            print(f"  Min:     {df_merged[col].min():,.2f}")
            print(f"  Max:     {df_merged[col].max():,.2f}")
            print(f"  Mean:    {df_merged[col].mean():,.2f}")
            print(f"  Latest:  {df_merged[col].iloc[-1]:,.2f} ({df_merged['date'].iloc[-1].strftime('%Y-%m-%d')})")
else:
    print("No data for summary")

## 💾 Export to CSV (Optional)

Uncomment the code below to save data to CSV file.

In [ ]:
# if not df_merged.empty:
#     filename = f"bond_{BOND_CODE}_{datetime.now().strftime('%Y%m%d')}.csv"
#     df_merged.to_csv(filename, index=False)
#     print(f"✅ Data exported to {filename}")

---

## Quick Bond Code Reference

Common bond codes for testing:
- `003720067` - 10-Year Bond #372
- `003730059` - 10-Year Bond #373
- `003740041` - 10-Year Bond #374

**To check a different bond:**
1. Go back to the "CHANGE BOND CODE HERE" cell
2. Change the BOND_CODE value
3. Run all cells again (Cell → Run All)